In [1]:
import time
import albumentations as A
import cv2
import numpy as np
import os
import requests
import urllib.request
import warnings
import json
import random
import shutil

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

warnings.filterwarnings("ignore")

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


# Daftar Gambar Yang Akan Di Download

In [3]:
daftar_gambar = meja_kerja_items = [
    "garbage on table",
    "trash on desk",
    "waste on table",
    "food waste on table",
    "fruit peel on table",
    "paper trash on desk",
    "discarded food on table",
    "waste paper and food on table",
]

# Membuat Directory

In [23]:
for item in daftar_gambar:
    folder_name = f'images\\{item}'
    if not os.path.isdir(folder_name):
        os.makedirs(folder_name)

# Download Gambar

In [24]:
def download_image(url, folder_name, num):
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(folder_name, str(num) + "_.jpg"), 'wb') as file:
            file.write(response.content)

def download_image_2(url, folder_name, num):
    response = urllib.request.urlopen(url)
    data = response.read()

    with open(os.path.join(folder_name, str(num) + "_.jpg"), 'wb') as file:
        file.write(data)

# Proses Download Gambar

In [25]:
chromeDriverPath = r"D:\Utils\compfest-scrapping\chromedriver-win64\chromedriver.exe"

options = webdriver.ChromeOptions()
service = Service(executable_path=chromeDriverPath)
driver = webdriver.Chrome(service=service, options=options)

base_url = 'https://www.google.com/search?q='

for item in daftar_gambar:

    folder_name = f'images\\{item}'

    keyword = "+" + "+".join(item.split(" "))

    driver.get(f"https://www.google.com/search?q={keyword}&tbm=isch")

    for i in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    driver.execute_script("window.scrollTo(document.body.scrollHeight, 0 );")

    page_html = driver.page_source
    page_soup = BeautifulSoup(page_html, "html.parser")

    div_container = page_soup.find_all("div", {"class": "eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd"})

    jumlah_data = len(div_container)

    for i in range(1, jumlah_data + 1):
        try:
            if i == 50:
                break
                
            if i % 25 == 0:
                continue
                
            xPath = f"""//*[@id="rso"]/div/div/div[1]/div/div/div[{i}]"""

            previewImageXPath = f"""//*[@id="rso"]/div/div/div[1]/div/div/div[{i}]/div[2]/h3/a/div/div/div/g-img/img"""
            previewImageElement = driver.find_element(By.XPATH, previewImageXPath)
            previewImageURL = previewImageElement.get_attribute("src")

            driver.find_element(By.XPATH, xPath).click()

            time.sleep(6)

            imageElement = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, """//*[@id="Sva75c"]/div[2]/div[2]/div/div[2]/c-wiz/div/div[3]/div[1]/a/img[1]"""))
            )
            imageURL = imageElement.get_attribute('src')

            try:
                download_image_2(imageURL, folder_name, i)
                print(f"Berhasil-foto-{i}")
            except Exception as e:
                print(f"Gagal-foto-{i}, error: {e}")

        except Exception as e:
            print(f"Gagal-foto-{i}, error: {e}")

driver.quit()

Berhasil-foto-1
Berhasil-foto-2
Berhasil-foto-3
Berhasil-foto-4
Berhasil-foto-5
Berhasil-foto-6
Berhasil-foto-7
Berhasil-foto-8
Berhasil-foto-9
Gagal-foto-10, error: HTTP Error 403: Forbidden
Gagal-foto-11, error: HTTP Error 406: Not Acceptable
Gagal-foto-12, error: HTTP Error 403: Forbidden
Berhasil-foto-13
Gagal-foto-14, error: HTTP Error 403: Forbidden
Berhasil-foto-15
Berhasil-foto-16
Berhasil-foto-17
Berhasil-foto-18
Berhasil-foto-19
Berhasil-foto-20
Berhasil-foto-21
Berhasil-foto-22
Gagal-foto-23, error: HTTP Error 403: Forbidden
Berhasil-foto-24
Berhasil-foto-26
Berhasil-foto-27
Berhasil-foto-28
Berhasil-foto-29
Berhasil-foto-30
Gagal-foto-31, error: HTTP Error 403: Forbidden
Berhasil-foto-32
Berhasil-foto-33
Berhasil-foto-34
Berhasil-foto-35
Berhasil-foto-36
Berhasil-foto-37
Berhasil-foto-38
Gagal-foto-39, error: HTTP Error 403: Forbidden
Berhasil-foto-40
Gagal-foto-41, error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer 

# **Augmentasi Data**

In [14]:
output_dir = r"D:\Utils\compfest-scrapping\images"

transform = A.Compose([
    A.Rotate(limit=45, p=1.0), 
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),  
    A.GaussianBlur(p=0.3),  
    A.Blur(blur_limit=3, p=0.2),  
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3), 
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5) 
])

In [5]:
for subdir in os.listdir(output_dir):
    if subdir in ["messy", "tidy"]:
        continue

    subdir_path = os.path.join(output_dir, subdir)
    list_of_images = os.listdir(subdir_path)

    for image_name in list_of_images:
        image_path = os.path.join(subdir_path, image_name)
        image = cv2.imread(image_path)
        if image is None:
            continue 

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        num_augmentations = 4

        for i in range(num_augmentations):
            augmented = transform(image=image_rgb)
            augmented_image = augmented['image']
            augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

            output_image_name = f'{os.path.splitext(image_name)[0]}_aug_{i+1}.jpg'
            output_image_path = os.path.join(subdir_path, output_image_name)

            cv2.imwrite(output_image_path, augmented_image_bgr)

In [13]:
output_dir = r"D:\Utils\compfest-scrapping\images"
subdir_path = os.path.join(output_dir, "messy power strip")
list_of_images = os.listdir(subdir_path)

for image_name in list_of_images:
    image_path = os.path.join(subdir_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    num_augmentations = 4

    for i in range(num_augmentations):
        augmented = transform(image=image_rgb)
        augmented_image = augmented['image']
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

        output_image_name = f'{os.path.splitext(image_name)[0]}_aug_{i+1}.jpg'
        output_image_path = os.path.join(subdir_path, output_image_name)
        cv2.imwrite(output_image_path, augmented_image_bgr)

# **Hapus Gambar yang Tidak Memiliki Label**

In [10]:
import os

main_directory = r"D:\Utils\compfest-scrapping\images\messy cable"
for image_cat in os.listdir(main_directory):
    image_cat_path = os.path.join(main_directory, image_cat)

    if os.path.isdir(image_cat_path):
        for image in os.listdir(image_cat_path):
            image_path = os.path.join(image_cat_path, image)

            json_file_name = f"{os.path.splitext(image)[0]}.json"
            json_file_path = os.path.join(image_cat_path, json_file_name)

            # Debug print statements
            print(f"Checking image: {image}")
            print(f"Image path: {image_path}")
            print(f"Expected JSON path: {json_file_path}")

            if not os.path.exists(json_file_path) and not image.endswith(".json"):
                print(f"Deleting image: {image_path}")
                os.remove(image_path)
            else:
                print(f"Skipping image: {image_path}")


In [5]:
source_directory = r"D:\Utils\compfest-scrapping\images"
destination_directory = r"D:\Utils\compfest-scrapping\images\images"

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

for root, dirs, files in os.walk(source_directory):
    for file in files:
        if not file.endswith(".json"):
            source_path = os.path.join(root, file)
            destination_path = os.path.join(destination_directory, file)
            shutil.move(source_path, destination_path)
            print(f"Memindahkan {file} ke {destination_directory}")


Memindahkan fo10_.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo10__aug_1.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo10__aug_2.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo10__aug_3.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo10__aug_4.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo11_.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo11__aug_1.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo11__aug_2.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo11__aug_3.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo11__aug_4.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo12_.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo12__aug_1.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo12__aug_2.jpg ke D:\Utils\compfest-scrapping\images\images
Memindahkan fo12__aug_3.jpg ke D:\Utils\compfest-scrapping\images\ima

# **Cek Daftar Label Yang Ada di Dalam Data**

In [48]:
daftar_label = []

for image_cat in os.listdir(main_directory):
    image_cat_path = os.path.join(main_directory, image_cat)

    if os.path.isdir(image_cat_path):
        for data in os.listdir(image_cat_path):
            data_path = os.path.join(image_cat_path, data)
            
            if data.endswith(".json"):
                with open(data_path, 'r') as json_file:
                    data = json.load(json_file)

                label = data['shapes'][0]['label']    
                if label not in daftar_label:
                    daftar_label.append(label)

# **Membuat Kelas Untuk Gambar**

In [49]:
kelas_gambar = {}
for indeks, label_cat in enumerate(daftar_label):
    kelas_gambar[label_cat] = indeks 

# **Konversi Data JSON**

In [55]:
def convert_json_to_yolo(json_data, output_dir, label_to_class_id, DIR):
    image_width = json_data["imageWidth"]
    image_height = json_data["imageHeight"]
    image_path = json_data["imagePath"]

    yolo_annotations = []

    for shape in json_data["shapes"]:
        x_min, y_min = shape["points"][0]
        x_max, y_max = shape["points"][1]

        x_center = (x_min + x_max) / 2 / image_width
        y_center = (y_min + y_max) / 2 / image_height
        width = (x_max - x_min) / image_width
        height = (y_max - y_min) / image_height

        class_id = label_to_class_id.get(shape["label"], None)

        if class_id is not None:
            yolo_annotation = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            yolo_annotations.append(yolo_annotation)

    output_file_name = f"{DIR}-{os.path.splitext(os.path.basename(image_path))[0]}.txt"
    output_file_path = os.path.join(output_dir, output_file_name)

    with open(output_file_path, 'w') as f:
        f.write("\n".join(yolo_annotations))

# **Data Validation**

In [57]:
main_directory = r"D:\Utils\compfest-scrapping\Model-2"
val_dir = r"D:\Utils\compfest-scrapping\0-dataset\labels\val"
new_place = r"D:\Utils\compfest-scrapping\0-dataset\images\val"

for image_cat in os.listdir(main_directory):
    image_cat_path = os.path.join(main_directory, image_cat)

    if os.path.isdir(image_cat_path):
        files = [f for f in os.listdir(image_cat_path) if not f.endswith(".json") and "aug" not in f]
        total_files = len(files)
        num_files_to_process = int(total_files * 0.20)

        files_to_process = random.sample(files, min(num_files_to_process, total_files))

        for data in files_to_process:
            data_path = os.path.join(image_cat_path, data)
            label_file = f"{os.path.splitext(data)[0]}.json"
            label_file_path = os.path.join(image_cat_path, label_file)

            if os.path.exists(label_file_path):
                with open(label_file_path, 'r') as json_file:
                    data_json = json.load(json_file)

                convert_json_to_yolo(data_json, val_dir, kelas_gambar, image_cat)

                new_file_name = f"{image_cat}_{data}"
                new_file_path = os.path.join(new_place, new_file_name)

                shutil.move(data_path, new_file_path)
            else:
                print(f"Label file not found: {label_file_path}")


# **Data Train**

In [58]:
main_directory = r"D:\Utils\compfest-scrapping\Model-2"
val_dir = r"D:\Utils\compfest-scrapping\0-dataset\labels\train"
new_place = r"D:\Utils\compfest-scrapping\0-dataset\images\train"

for image_cat in os.listdir(main_directory):
    image_cat_path = os.path.join(main_directory, image_cat)

    if os.path.isdir(image_cat_path):
        files = [f for f in os.listdir(image_cat_path) if not f.endswith(".json")]

        for data in files:
            data_path = os.path.join(image_cat_path, data)
            label_file = f"{os.path.splitext(data)[0]}.json"
            label_file_path = os.path.join(image_cat_path, label_file)

            if os.path.exists(label_file_path):
                with open(label_file_path, 'r') as json_file:
                    data_json = json.load(json_file)

                convert_json_to_yolo(data_json, val_dir, kelas_gambar, image_cat)

                new_file_name = f"{image_cat}_{data}"
                new_file_path = os.path.join(new_place, new_file_name)

                shutil.move(data_path, new_file_path)
            else:
                print(f"Label file not found: {label_file_path}")

In [62]:
import os

def check_labels(image_dir, label_dir):
    images = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    labels = [f.replace('.txt', '') for f in os.listdir(label_dir) if f.endswith('.txt')]

    missing_labels = []

    for image in images:
        image_name = os.path.splitext(image)[0]

        if image_name not in labels:
            missing_labels.append(image)

    if missing_labels:
        print("The following images are missing labels:")
        for img in missing_labels:
            print(img)
    else:
        print("All images have corresponding labels.")

image_dir = r"D:\Utils\compfest-scrapping\dataset\train\images"
label_dir = r"D:\Utils\compfest-scrapping\dataset\train\labels"

check_labels(image_dir, label_dir)


The following images are missing labels:
Adaptor_10_.jpg
Adaptor_10__aug_1.jpg
Adaptor_10__aug_2.jpg
Adaptor_10__aug_3.jpg
Adaptor_10__aug_4.jpg
Adaptor_11_.jpg
Adaptor_11__aug_1.jpg
Adaptor_11__aug_2.jpg
Adaptor_11__aug_3.jpg
Adaptor_11__aug_4.jpg
Adaptor_13_.jpg
Adaptor_13__aug_1.jpg
Adaptor_13__aug_2.jpg
Adaptor_13__aug_3.jpg
Adaptor_13__aug_4.jpg
Adaptor_14_.jpg
Adaptor_14__aug_1.jpg
Adaptor_14__aug_2.jpg
Adaptor_14__aug_3.jpg
Adaptor_14__aug_4.jpg
Adaptor_15__aug_1.jpg
Adaptor_15__aug_2.jpg
Adaptor_15__aug_3.jpg
Adaptor_15__aug_4.jpg
Adaptor_16_.jpg
Adaptor_16__aug_1.jpg
Adaptor_16__aug_2.jpg
Adaptor_16__aug_3.jpg
Adaptor_16__aug_4.jpg
Adaptor_17__aug_1.jpg
Adaptor_17__aug_2.jpg
Adaptor_17__aug_3.jpg
Adaptor_17__aug_4.jpg
Adaptor_18_.jpg
Adaptor_18__aug_1.jpg
Adaptor_18__aug_2.jpg
Adaptor_18__aug_3.jpg
Adaptor_18__aug_4.jpg
Adaptor_19_.jpg
Adaptor_19__aug_1.jpg
Adaptor_19__aug_2.jpg
Adaptor_19__aug_3.jpg
Adaptor_19__aug_4.jpg
Adaptor_1_.jpg
Adaptor_1__aug_1.jpg
Adaptor_1__aug_2.jp

In [2]:
for dir in os.listdir(r"D:\Utils\compfest-scrapping\images"):
    sub_dir = os.path.join(r"D:\Utils\compfest-scrapping\images", dir)

    if os.path.isdir(sub_dir):  
        for data in os.listdir(sub_dir):
            old = os.path.join(sub_dir, data)
            new = os.path.join(sub_dir, dir[0:2] + data)
            os.rename(old, new)

In [17]:
directory = r"D:\Utils\compfest-scrapping\Model-4\dataset\labels"

def replace_label_in_json(file_path, old_label, new_label):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for shape in data.get('shapes', []):
        if shape['label'] == old_label:
            print(f"Replacing label '{old_label}' with '{new_label}' in file: {file_path}")
            shape['label'] = new_label

    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            replace_label_in_json(file_path, "anomali_dokumen", "anomali")

Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\11.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\13.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\13.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\13.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\14.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\19.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\19_.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\Utils\compfest-scrapping\Model-4\dataset\labels\19_.json
Replacing label 'anomali_dokumen' with 'anomali' in file: D:\U

In [3]:
transform = A.Compose([
    A.Rotate(limit=90, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.5),
    A.Blur(blur_limit=3, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
])

output_dir = r"D:\Utils\compfest-model\image"
subdir_path = os.path.join(output_dir, "test-gambar")
list_of_images = os.listdir(subdir_path)

for image_name in list_of_images:
    image_path = os.path.join(subdir_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    num_augmentations = 5

    for i in range(num_augmentations):
        augmented = transform(image=image_rgb)
        augmented_image = augmented['image']
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

        output_image_name = f'{os.path.splitext(image_name)[0]}_aug_{i+1}.jpg'
        output_image_path = os.path.join(subdir_path, output_image_name)
        cv2.imwrite(output_image_path, augmented_image_bgr)

In [9]:
main_path = r"D:\Utils\compfest-scrapping\Model-2\images"

for foto in os.listdir(main_path):
    category = foto.split(sep='_')[0]

    old_path = os.path.join(main_path, foto)
    new_directory = os.path.join(r"D:\Utils\compfest-scrapping\Model-2", category)

    os.makedirs(new_directory, exist_ok=True)

    new_path = os.path.join(new_directory, foto)

    shutil.copy(old_path, new_path)

    print(f"Copied {foto} to {new_path}")

Copied Adaptor_10_.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_10_.jpg
Copied Adaptor_10__aug_1.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_10__aug_1.jpg
Copied Adaptor_10__aug_2.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_10__aug_2.jpg
Copied Adaptor_10__aug_3.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_10__aug_3.jpg
Copied Adaptor_10__aug_4.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_10__aug_4.jpg
Copied Adaptor_11_.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_11_.jpg
Copied Adaptor_11__aug_1.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_11__aug_1.jpg
Copied Adaptor_11__aug_2.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_11__aug_2.jpg
Copied Adaptor_11__aug_3.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_11__aug_3.jpg
Copied Adaptor_11__aug_4.jpg to D:\Utils\compfest-scrapping\Model-2\Adaptor\Adaptor_11__aug_4.jpg
Copied Adaptor_13_.jpg to D:\Utils\compfest-

In [12]:
main_path = r"D:\Utils\compfest-scrapping\Model-2"

for category in os.listdir(main_path):
    cat_path = os.path.join(main_path, category)

    if os.path.isdir(cat_path):  
        for file in os.listdir(cat_path):

            if file.endswith(".json"):
                old_file_path = os.path.join(cat_path, file)

                new_file_name = f"{category}_{file}" 
                new_file_path = os.path.join(cat_path, new_file_name)  

                os.rename(old_file_path, new_file_path)

In [13]:
main_path = r"D:\Utils\compfest-scrapping\Model-2"
labels_path = os.path.join(main_path, "labels")
images_path = os.path.join(main_path, "images")

# Create 'labels' and 'images' directories if they do not exist
os.makedirs(labels_path, exist_ok=True)
os.makedirs(images_path, exist_ok=True)

for category in os.listdir(main_path):
    cat_path = os.path.join(main_path, category)

    if os.path.isdir(cat_path):  # Ensure it's a directory
        for file in os.listdir(cat_path):
            old_file_path = os.path.join(cat_path, file)

            if file.endswith(".json"):
                new_file_path = os.path.join(labels_path, file)
            else:
                new_file_path = os.path.join(images_path, file)

            shutil.move(old_file_path, new_file_path)

In [ ]:
output_dir = r"D:\Utils\compfest-model\image"
subdir_path = os.path.join(output_dir, "test-gambar")
list_of_images = os.listdir(subdir_path)

for image_name in list_of_images:
    image_path = os.path.join(subdir_path, image_name)
    image = cv2.imread(image_path)
    if image is None:
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    num_augmentations = 5

    for i in range(num_augmentations):
        augmented = transform(image=image_rgb)
        augmented_image = augmented['image']
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

        output_image_name = f'{os.path.splitext(image_name)[0]}_aug_{i+1}.jpg'
        output_image_path = os.path.join(subdir_path, output_image_name)
        cv2.imwrite(output_image_path, augmented_image_bgr)